# RAG vs SQL Agent: E-commerce Customer Support

This notebook compares Retrieval-Augmented Generation (RAG) and SQL Agent approaches to enable natural language queries over e-commerce support data.

## 1. Technical Architectures

**RAG:**
- Embeds all database content into a vector store (ChromaDB)
- Queries use dense vector search + LLM

**SQL Agent:**
- Uses LangChain SQL agent with SQLite to parse and generate SQL queries

In [ ]:
# SQLite Setup: Create and populate sample tables
import sqlite3
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY, name TEXT, email TEXT
);""")
cursor.execute("INSERT INTO customers (name, email) VALUES ('Alice', 'alice@example.com');")
conn.commit()

In [ ]:
# SQL Agent Setup
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI

db = SQLDatabase.from_uri("sqlite:///ecommerce.db")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
agent_executor = create_sql_agent(llm=llm, db=db, agent_type="openai-tools")
# Example: agent_executor.invoke({"input": "Show all customers"})

In [ ]:
# RAG + Chroma Setup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

docs = [Document(page_content="Alice placed an order for headphones"),
        Document(page_content="Bob wrote a review for the laptop")]
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

## 2. Streamlit UI for Querying
- A simple natural language interface to choose SQL Agent or RAG

In [ ]:
# Save this block as app.py to run
import streamlit as st

st.title("Customer Support: RAG vs SQL Agent")
query = st.text_input("Ask a question:")
mode = st.radio("Choose method", ["SQL Agent", "RAG"])

if st.button("Run"):
    if mode == "SQL Agent":
        result = agent_executor.invoke({"input": query})
        st.write(result["output"])
    else:
        from langchain.chains import RetrievalQA
        qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
        result = qa_chain.run(query)
        st.write(result)

## 3. Performance Benchmark
Simulated results comparing latency, accuracy, and resources.

| Query | RAG Time | SQL Agent Time | Accuracy |
|-------|----------|----------------|----------|
| What did Alice buy? | 1.5s | 0.7s | ✅ Both |
| List all reviews | 1.8s | 0.6s | ✅ SQL |
| Order insights summary | ✅ | ❌ | ✅ RAG |

## 4. Recommendation Matrix

| Scenario | RAG | SQL Agent |
|----------|-----|------------|
| Natural queries | ✅ | ❌ |
| Filtered queries | ❌ | ✅ |
| Summarization | ✅ | ❌ |
| Data joins | ❌ | ✅ |